In [2]:
import pandas as pd
import numpy as np
import math
import ast

import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(1337)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
test_data = pd.read_csv('test_data.csv')
train_data = pd.read_csv('train_data.csv')
valid_data = pd.read_csv('valid_data.csv')
word_vectors = pd.read_csv('wv.csv')

In [5]:
dictionary = {}
dictionary_size = word_vectors['vectors'].shape[0]

for i in range(dictionary_size):
    wv = np.fromstring(word_vectors['vectors'][i][1:-1], sep=' ')
    dictionary[word_vectors['word'][i]] = torch.from_numpy(wv)

In [71]:
d_model = 64
num_heads = 4
batch_size = 30
max_sequence_length = 64
ffn_hidden = 256
num_layers = 4
learning_rate = 1e-3
max_iters = 10000
eval_interval = 500
eval_iters = 10

block_size = 10
batch_size = 32

In [72]:
X_train = []
y_train = []

X_test = []
y_test = []

X_valid = []
y_valid = []

for i in range(len(train_data)):
    tokens = ast.literal_eval(train_data['tokens'][i])
    pos_tags = ast.literal_eval(train_data['pos_tags'][i])
    X = []
    y = []

    for j in range(len(tokens)):
        if (tokens[j] not in dictionary):
            X += [torch.zeros(64)]
        else:
            X += [dictionary[tokens[j]]]
        y += [torch.tensor(pos_tags[j])]
    if len(tokens) < block_size:
        for j in range(block_size-len(tokens)):
            X += [torch.zeros(64)]
            y += [torch.tensor(45)]

    X_train.append(X)
    y_train.append(y)

for i in range(len(valid_data)):
    tokens = ast.literal_eval(valid_data['tokens'][i])
    pos_tags = ast.literal_eval(valid_data['pos_tags'][i])
    X = []
    y = []

    for j in range(len(tokens)):
        if (tokens[j] not in dictionary):
            X += [torch.zeros(64)]
        else:
            X += [dictionary[tokens[j]]]
        y += [torch.tensor(pos_tags[j])]
    if len(tokens) < block_size:
        for j in range(block_size-len(tokens)):
            X += [torch.zeros(64)]
            y += [torch.tensor(45)]

    X_valid.append(X)
    y_valid.append(y)

for i in range(len(test_data)):
    tokens = ast.literal_eval(test_data['tokens'][i])
    pos_tags = ast.literal_eval(test_data['pos_tags'][i])
    X = []
    y = []

    for j in range(len(tokens)):
        if (tokens[j] not in dictionary):
            X += [torch.zeros(64)]
        else:
            X += [dictionary[tokens[j]]]
        y += [torch.tensor(pos_tags[j])]
    if len(tokens) < block_size:
        for j in range(block_size-len(tokens)):
            X += [torch.zeros(64)]
            y += [torch.tensor(45)]

    X_test.append(X)
    y_test.append(y)

In [78]:
def get_batch(split):
    xtemp = []
    ytemp = []

    if split == 'train':
        X, Y = X_train, y_train
    elif split == 'valid':
        X, Y = X_valid, y_valid
    else:
        X, Y = X_test, y_test

    ix = torch.randint(len(X), (batch_size, ))
    for i in ix:
        if len(X[i]) > block_size:
            j = torch.randint(len(X[i]) - block_size, (1, ))
            xtemp.append(torch.stack(X[i][j:j+block_size]))
            ytemp.append(torch.stack(Y[i][j:j+block_size]))
        else:
            xtemp.append(torch.stack(X[i]))
            ytemp.append(torch.stack(Y[i]))

    x, y = torch.stack(xtemp), torch.stack(ytemp)
    x, y = x.type(torch.FloatTensor), y
    x, y = x.to(device), y.to(device)
    return x, y

In [79]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    accuracy = {}
    model.eval()
    for split in ['train', 'valid']:
        losses = torch.zeros(eval_iters)
        total = 0
        correct = 0
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()

            N, _ = logits.shape
            Y = Y.view(N)
            total += N
            for i in range(N):
                probs = F.softmax(logits[i], -1)
                idx = torch.multinomial(probs, num_samples=1)
                if (idx in [21, 22, 23, 24, 25, 28, 29] and Y[i] in [21, 22, 23, 24, 25, 28, 29]):
                    correct += 1
                elif (idx in [37, 38, 39, 40, 41, 42] and Y[i] in [37, 38, 39, 40, 41, 42]):
                    correct += 1
                elif (idx in [16, 17, 18, 30, 31, 32] and Y[i] in [16, 17, 18, 30, 31, 32]):
                    correct += 1
                elif (idx not in [21, 22, 23, 24, 25, 28, 29, 37, 38, 39, 40, 41, 42, 16, 17, 18, 30, 31, 32] and Y[i] not in [21, 22, 23, 24, 25, 28, 29, 37, 38, 39, 40, 41, 42, 16, 17, 18, 30, 31, 32]):
                    correct += 1

        out[split] = losses.mean()
        accuracy[split] = (float(correct)/total)
    model.train()
    return out, accuracy

In [83]:
class Head(nn.Module):
    def __init__(self, embd_size, num_heads):
        super().__init__()
        self.query = nn.Linear(embd_size, embd_size // num_heads, bias=False)
        self.key = nn.Linear(embd_size, embd_size // num_heads, bias=False)
        self.value = nn.Linear(embd_size, embd_size // num_heads, bias=False)

    def forward(self, x):
        _, _, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-1, -2) / (C ** 0.5)
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out

class MultiheadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head(d_model, num_heads) for _ in range(num_heads)])
        self.proj = nn.Linear(d_model, d_model)

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x

class LayerNorm(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y  + self.beta
        return out


class FeedForward(nn.Module):
    def __init__(self, d_model, hidden):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, hidden),
            nn.Tanh(),
            nn.Linear(hidden, d_model),
        )

    def forward(self, x):
        return self.net(x)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len: int = 5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = self.pe[:x.size(1)]
        return x


class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads):
        super(EncoderLayer, self).__init__()
        self.attention = MultiheadAttention(d_model=d_model, num_heads=num_heads)
        self.ffn = FeedForward(d_model=d_model, hidden=ffn_hidden)
        self.norm1 = LayerNorm(parameters_shape=[d_model])
        self.norm2 = LayerNorm(parameters_shape=[d_model])

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ffn(self.norm2(x))
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, num_layers, block_size, d_input, d_output):
        super().__init__()
        self.position_embedding_table = PositionalEncoding(d_model, max_len = block_size)
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads) for _ in range(num_layers)])
        self.norm = LayerNorm(parameters_shape=[d_model])
        self.linear = nn.Linear(d_model, d_output)

    def forward(self, x, target=None):
        embds = x + self.position_embedding_table(x)
        embds = self.layers(embds)
        embds = self.norm(embds)
        logits = self.linear(embds)

        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss

In [84]:
encoder = Encoder(d_model, ffn_hidden, num_heads, num_layers, block_size, 64, 47)
encoder = encoder.to(device)
optimizer = torch.optim.AdamW(encoder.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
print(sum(p.numel() for p in encoder.parameters())/1e6, 'M parameters')

0.202351 M parameters


In [88]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses, accuracy = estimate_loss(encoder)
        print(f"step {iter}: train loss {losses['train']:.4f} accuracy: {accuracy['train']:.4f} valid loss {losses['valid']:.4f} accuracy: {accuracy['valid']:.4f}")

    Xb, yb = get_batch('train')
    _, loss = encoder(Xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.2332 accuracy: 0.9322 valid loss 0.4465 accuracy: 0.9113
step 500: train loss 0.2566 accuracy: 0.9403 valid loss 0.4412 accuracy: 0.9069
step 1000: train loss 0.2249 accuracy: 0.9387 valid loss 0.3962 accuracy: 0.9087
step 1500: train loss 0.2285 accuracy: 0.9300 valid loss 0.4831 accuracy: 0.9044
step 2000: train loss 0.2111 accuracy: 0.9375 valid loss 0.4484 accuracy: 0.9056
step 2500: train loss 0.2035 accuracy: 0.9409 valid loss 0.3806 accuracy: 0.9241
step 3000: train loss 0.2016 accuracy: 0.9425 valid loss 0.4952 accuracy: 0.8972
step 3500: train loss 0.2198 accuracy: 0.9325 valid loss 0.4471 accuracy: 0.9050
step 4000: train loss 0.2266 accuracy: 0.9322 valid loss 0.4538 accuracy: 0.9069
step 4500: train loss 0.2172 accuracy: 0.9394 valid loss 0.4184 accuracy: 0.9131
step 5000: train loss 0.2192 accuracy: 0.9425 valid loss 0.4295 accuracy: 0.9125
step 5500: train loss 0.2171 accuracy: 0.9425 valid loss 0.4264 accuracy: 0.9150
step 6000: train loss 0.1739 acc